# اتصال به گوگل کولب

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# خواندن تصاویر

In [ ]:
import cv2
import numpy as np
import glob
from skimage import io

width=200
height=200


Data=[]
Target=[]

# glioma_tumor
file_list = glob.glob(r'/content/drive/My Drive/Brain Tumor/Training/glioma_tumor/*.jpg')

for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(0)

file_list = glob.glob(r'/content/drive/My Drive/Brain Tumor/Training/meningioma_tumor/*.jpg')
# meningioma_tumor
for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(1)

# pituitary_tumor
file_list = glob.glob(r'/content/drive/My Drive/Brain Tumor/Training/pituitary_tumor/*.jpg')
for i in file_list:
  im=cv2.imread(i)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(width,height))
  Data.append(im)
  Target.append(2)



# پیش پردازش

In [ ]:
from tensorflow.keras.utils import to_categorical
Final_Target = to_categorical(Target)

In [ ]:
Final_Data=np.reshape(Data,(len(Data),height,width,1))

# استخراج ویژگی

In [ ]:
# CNN LSTM

In [ ]:
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.models import Model

myInput=layers.Input(shape=(height,width,1))
conv1=SeparableConv2D(64,3, activation='relu',padding='same')(myInput)
max1=MaxPooling2D(2)(conv1)
conv2=SeparableConv2D(64,3, activation='relu',padding='same')(max1)
max2=MaxPooling2D(2)(conv2)
Time1=layers.TimeDistributed(Flatten())(max2)
lstm1=LSTM(150)(Time1)
des1 =Dense(50, activation='relu')(lstm1)
Extract_Model = Model(myInput, des1)
des2 =Dense(20, activation='relu')(des1)
out_layer =Dense(3, activation='softmax')(des2)
model = Model(myInput, out_layer)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

verbose, epochs, batch_size = 1, 10, 32
model.fit(Final_Data,Final_Target, epochs=epochs, batch_size=batch_size, verbose=verbose)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200, 200, 1)]     0         
                                                                 
 separable_conv2d_4 (Separab  (None, 200, 200, 64)     137       
 leConv2D)                                                       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 100, 100, 64)     0         
 2D)                                                             
                                                                 
 separable_conv2d_5 (Separab  (None, 100, 100, 64)     4736      
 leConv2D)                                                       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 50, 50, 64)       0         
 2D)                                                       

# جداسازی داده

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data, Target, test_size=0.3, random_state=42)

In [ ]:
from tensorflow.keras.utils import to_categorical
trainy = to_categorical(y_train)
testy = to_categorical(y_test)

In [ ]:
trainX=np.reshape(X_train,(len(X_train),height,width,1))
testX=np.reshape(X_test,(len(X_test),height,width,1))

In [ ]:
trainFeature=(Extract_Model.predict(trainX))
testFeature=(Extract_Model.predict(testX))

# طبقه بندی

In [ ]:
from sklearn import svm
classifier_svm = svm.LinearSVC()
classifier_svm.fit(trainFeature, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC()

In [ ]:
from sklearn import tree
classifier_tree = tree.DecisionTreeClassifier()
classifier_tree.fit(trainFeature, y_train)

DecisionTreeClassifier()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier_Knn = KNeighborsClassifier(n_neighbors=3)
classifier_Knn.fit(trainFeature, y_train)

KNeighborsClassifier(n_neighbors=3)

# ترکیب طبقه بندی

In [ ]:
from sklearn.ensemble import  VotingClassifier
classifier_Voting = VotingClassifier(estimators=[('SVM', classifier_svm), ('TREE', classifier_tree), ('KNN', classifier_Knn)],weights=[1,1,1])
classifier_Voting.fit(trainFeature, y_train)
Class_Voting=classifier_Voting.predict(testFeature)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


# نتایج

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test,Class_Voting))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       248
           1       0.99      0.99      0.99       248
           2       1.00      0.99      0.99       247

    accuracy                           0.99       743
   macro avg       0.99      0.99      0.99       743
weighted avg       0.99      0.99      0.99       743



In [ ]:
print('Accuracy= ' + str(round(accuracy_score(y_test, Class_Voting)*100)))

Accuracy= 99
